In [2]:
%matplotlib inline
import os
from IPython.display import Image
from IPython.display import display
import numpy as np
import pandas as pd
from scipy import interp
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import auc, confusion_matrix,  f1_score, precision_score, recall_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from cycler import cycler
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
plt = matplotlib.pyplot

In [3]:
from glob import glob
files = glob("player_data/*.csv")

In [4]:
pd.read_csv(files[6])


,Unnamed: 0,Age,Squad,Country,Comp,LgRank,90s,Cmp,Att,Cmp%,...,PrgDist.1,Prog.1,1/3.1,CPA,Mis,Dis,Targ,Rec,Rec%,Prog.2
0,0,37,Chelsea,eng ENG,1. Premier League,4th,5.0,121,165,73.3,...,208,0,0,0,0,0,110,110,100.0,0
1,1,38,Chelsea,eng ENG,1. Premier League,4th,1.0,24,26,92.3,...,20,0,0,0,0,0,15,15,100.0,0


In [5]:
len(files)

532

dfxx = pd.read_csv(files[532]) ## out of range

dfxx

In [6]:
def one_player(df_one):
    df_num=df_one.shape[0]
    one_player=np.zeros((1, 71))
    for i in range(0,df_num):
    #print(i)
        if df_one.values[i,4]=='1. Premier League' :  # only collect the Premier Leage data
            dfxx= df_one.values[i,6:]           # collect the data after the column "LgRank"
            one_player=np.row_stack((one_player,dfxx))
    one_player=np.delete(one_player, 0, 0)     # delete temporay row
    one_player=np.delete(one_player, 71-15, 1) # move the column have NAN
    one_player=np.mean(one_player,axis=0)      # average the data for different seasons
    return one_player

one_player(dfxx).shape

In [7]:
player= np.zeros((1, 70))  #70 becuase that i delelete one NAN column
for i in range(0,532):
    df_one = pd.read_csv(files[i])
    player_one= one_player(df_one)
    player= np.row_stack((player,player_one))
player=np.delete(player, 0, 0)     # delete temporay row

In [8]:
player.shape

(532, 70)

In [9]:
X=player
X=X.astype(np.float32)

發現還有missing value所以作以下處理

In [10]:
id=np.argwhere(np.isnan(X))
id2=id[:,1] # 有那些feature還有missing value  # of missing:132 
id3=id[:,0] # 含missing value的player

In [11]:
np.unique(id2)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
      dtype=int64)

In [12]:
XX=np.delete(X, id3, 0)   # delete players who have missing values

In [13]:
XX.shape

(494, 70)

KMEANS

In [14]:
sc = StandardScaler()
sc.fit(XX)
XX_train_std = sc.transform(XX)


In [15]:
from sklearn.cluster import KMeans


In [19]:
kmeans = KMeans(n_clusters=8, random_state=0).fit(XX_train_std)

In [20]:
kmeans.labels_

array([4, 3, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 0, 6, 3, 0, 2, 0, 0, 6, 7, 3,
       0, 0, 3, 3, 1, 3, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 7, 2, 4, 2, 7,
       0, 3, 3, 7, 3, 7, 3, 7, 3, 1, 3, 7, 3, 7, 7, 7, 1, 1, 7, 0, 4, 5,
       5, 7, 7, 2, 0, 2, 7, 0, 7, 3, 7, 2, 1, 7, 3, 7, 1, 3, 7, 1, 3, 1,
       4, 5, 0, 7, 3, 7, 7, 7, 0, 3, 7, 1, 0, 7, 0, 5, 5, 0, 1, 1, 1, 7,
       1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 2, 2, 3, 4, 5, 2, 0, 1, 2, 5, 0,
       7, 3, 3, 7, 3, 7, 1, 1, 1, 1, 1, 1, 1, 1, 0, 5, 3, 4, 6, 0, 2, 7,
       3, 7, 7, 5, 7, 7, 6, 3, 7, 1, 3, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 4,
       2, 5, 5, 0, 0, 7, 0, 5, 2, 5, 1, 3, 3, 1, 7, 5, 1, 1, 1, 1, 1, 1,
       0, 2, 3, 4, 7, 3, 3, 7, 1, 7, 7, 3, 0, 0, 3, 1, 7, 7, 1, 1, 7, 4,
       1, 1, 0, 3, 1, 1, 1, 1, 4, 0, 2, 0, 4, 5, 5, 6, 7, 0, 3, 7, 7, 1,
       1, 7, 3, 3, 1, 7, 1, 1, 1, 1, 1, 1, 7, 1, 4, 7, 5, 5, 0, 3, 2, 3,
       3, 3, 7, 2, 1, 1, 7, 3, 1, 1, 3, 1, 1, 1, 1, 6, 0, 3, 1, 4, 3, 7,
       2, 7, 3, 7, 5, 7, 4, 3, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
#kmeans.predict([[0, 0], [12, 3]])
#kmeans.cluster_centers_


In [21]:
from sklearn.mixture import GaussianMixture


In [22]:
gm = GaussianMixture(n_components=8, random_state=0).fit(XX_train_std)


In [24]:
#gm.means_
#gm.predict([[0, 0], [12, 3]])

In [25]:
from sklearn.cluster import DBSCAN


In [26]:
clustering = DBSCAN(eps=3, min_samples=2).fit(XX_train_std)
clustering.labels_


array([ 0,  1, -1,  2,  0,  2,  2,  2,  2,  2,  2, -1, -1, -1, -1, -1, -1,
       -1, -1, -1,  2, -1,  3, -1, -1,  4,  2,  5,  2, -1,  2,  2,  2,  2,
       -1,  2,  2,  2,  2, -1, -1,  0, -1, -1, -1, -1, -1,  2, -1,  2,  2,
        2, -1,  2,  2,  2,  6,  2, -1,  2,  2,  2, -1, -1,  0, -1, -1,  2,
       -1, -1,  7,  8,  2, -1,  2, -1,  9,  8,  2, -1,  2, 10,  2, -1, -1,
        2, -1, -1,  0, -1, -1,  2, -1, -1, -1,  2, -1,  4,  2,  2, -1,  2,
       -1, -1, -1, -1,  2,  2,  2,  2,  2,  2,  2,  2,  0,  2,  2,  2,  2,
        2,  2, -1, -1, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1,  2,
       11,  2,  2,  2, -1,  2,  2,  2,  2,  2,  7, -1, -1, -1, -1, -1, -1,
        2, -1,  2,  2, -1,  2,  2, -1, -1,  2,  2,  2,  2,  2,  2,  2, -1,
        2, -1,  2,  2,  2,  0, -1, -1, -1, -1, -1,  2, -1, -1, -1, -1,  2,
       -1, -1,  2, -1, -1,  2,  2,  2, -1,  2,  2, -1, -1, -1,  0,  2, -1,
       -1,  2,  2, -1,  2, -1, -1,  3, -1,  2,  2, -1,  2,  2,  2,  0,  2,
        2, -1, -1,  2,  2

In [28]:
from sklearn.neighbors import KernelDensity


In [29]:
import numpy as np

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(XX_train_std)
#kde.score_samples(X)
